# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Damarys Valenzuela Santos  |
| **Fecha**      | 15/09/2025  |
| **Expediente** |  744494 |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Ya que necesitamos dividir los datos en entrenamiento y prueba para evaluar si el modelo realmente generaliza a datos nuevos; entrenar y evaluar con los mismos datos puede llevar a un sobreajuste (el modelo “memoriza” en lugar de aprender). Con esta separación podemos medir su rendimiento real, ajustar mejor los hiperparámetros y simular cómo se comportará en escenarios reales, asegurándonos de que no solo funcione bien con los datos de entrenamiento, sino también con información desconocida.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Necesitamos reservar una parte como conjunto de prueba paa estimar la capacidad de generalización del modelo; si entrenamos con el 100% de los datos, no podríamos saber cómo se comportará ante información nueva.  

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")

In [ ]:
df= df.drop(columns="model")

In [ ]:
x=df.drop(columns="mpg")
y=df["mpg"]

In [ ]:
mses=[]
for i in range(32):
  x_train=x.drop(i)
  y_train=y.drop(i)
  x_test=x.iloc[[i]]
  y_test=y.iloc[[i]]

  modelo=LinearRegression()
  modelo.fit(x_train,y_train)

  y_train_pred=modelo.predict(x_train)
  y_test_pred=modelo.predict(x_test)

  error = mean_squared_error(y_test,y_test_pred)
  mses.append(error)

In [ ]:
np.mean(mses)

np.float64(12.181558006901943)

In [ ]:
np.std(mses)

np.float64(17.067399871888522)

Interpreta.

Con esto pudimos obtener una medida más robusta del rendimiento y podemos reducir el sesgo en la evaluación, aprovecha al máximo los datos de entrenamiento.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [30]:
k = 10
indices = np.arange(len(X))
np.random.shuffle(indices)
folds = np.array_split(indices, k)
for i in range(k):
    test_index = folds[i]
    train_index = np.hstack([folds[j] for j in range(k) if j != i])
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]

    modelo = LinearRegression()
    modelo.fit(x_train, y_train)

    y_test_pred = modelo.predict(x_test)

    error = mean_squared_error(y_test, y_test_pred)
    mses.append(error)

print(np.mean(mses))
print(np.std(mses))

0.5298501855992767
0.027647170869278107


Interpreta.

En promedio, el error de predicción es relativamente bajo y además bastante estable entre los diferentes folds, que muestra que el modelo es estable y consistente en las diferentes particiones de validación cruzada.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3